In [ ]:
import sqlite3
import pandas as pd
import numpy as np


In [2]:
db_path = r"E:\Desktop\webAutomation\job_agent.db"
conn = sqlite3.connect(db_path)

In [3]:

df = pd.read_sql_query("SELECT sentence,importance_label,confidence FROM job_sentences;", conn)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df = df.sample(frac=1, random_state=32).reset_index(drop=True)
print(df['importance_label'].value_counts())
df.head(20)

importance_label
0    55082
1    31462
Name: count, dtype: int64


,sentence,importance_label,confidence
0,Technology for today and tomorrow,0,None
1,"Here, ambition meets action",0,None
2,00 PM/1 PM/2PM IST onwards depending on Daylig...,1,None
3,Ensure timely and accurate processing and sett...,1,None
4,"Maintain updated records, files, and process d...",1,None
5,What You’ll Do (3 to 5 brief pointers about th...,0,None
6,For residential sites candidates with resident...,0,None
7,"Experience in computer vision, image, radar an...",1,None
8,Essential Skills,0,None
9,"At Salesforce, we are dedicated to fostering a...",1,None


In [ ]:
from joblib import Parallel, delayed

In [ ]:
corpus = []
import re
from normalizer.preprocess_data import process_sentence

corpus = Parallel(n_jobs=-1, backend="threading")(
    delayed(process_sentence)(df['sentence'][i]) for i in range(0,len(df['sentence'])) 
)

print(f"courpus-------")

corpus

{"didn't", 'your', 'will', 'myself', 'just', 'any', "needn't", 'then', 'about', "we'll", 'because', 'under', 'no', 'and', 'at', "i'll", 'over', 'themselves', "i'd", 'this', "wasn't", 'she', 'same', 'which', 'were', 'again', 'they', 'be', 'in', 'shan', 'shouldn', 'before', "don't", 'but', 'does', 'did', "mightn't", 'having', 'if', 'couldn', 'during', 'wasn', 'with', 'do', 'won', 'you', "you've", 't', 'or', 'don', 'off', 'some', "you'd", "i've", 'not', 'most', 'their', 'being', 'hers', "he'll", 'was', 'herself', 'until', 'all', 'why', "that'll", "shan't", "isn't", "weren't", 'can', 'hasn', 'aren', 'only', "hasn't", 'of', 'mightn', 'those', 'through', 'as', 'a', 'is', 'an', 're', 'the', 'who', 'am', 'more', 'than', 'few', 'for', "mustn't", 'our', 'ourselves', 'while', 'he', 'that', 'ain', 'has', 'so', 'me', 'when', "we'd", 'where', "they'd", "haven't", "shouldn't", 'her', 'll', "i'm", 'too', 'we', 'nor', 'between', "you're", 'them', "you'll", 'yourselves', "should've", 'yourself', 'theirs

['technolog today tomorrow',
 'here, ambit meet action',
 '00 pm/1 pm/2pm ist onward depend daylight save time',
 'ensur time accur process settlement domest lgc trade activ along regulatori report',
 'maintain updat records, files, process document per team guidelin',
 'what you’ll do (3 5 brief pointer role responsibility)',
 'for residenti site candid residenti experi prefer',
 'experi comput vision, image, radar lidar signal process',
 'essenti skill',
 'at salesforce, dedic foster divers inclus workplac individu background welcom valu',
 "the world work know chang look trailblaz passion better busi world ai, drive innovation, keep salesforce' core valu heart",
 'if need reason accommod disabl part employ process – includ onlin applic and/or overal select process – may email us hrscleaves@jl',
 'the base pay posit',
 "the main respons role vendor master creation & modification, invoic process payment process london stock exchang group' senior manag make well-inform decisions, lead 

In [9]:
# len(corpus)
df['sentence'][2]

'00 PM/1 PM/2PM IST onwards depending on Daylight Savings Time'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10500, min_df=4,max_df=0.65)
X=vectorizer.fit_transform(corpus).toarray()

In [11]:
from joblib import dump # for saving the tf-idf model permanent for no future retraining
dump(vectorizer,'tfidf_vectorizer.joblib')
print("TF-IDF vectorizer saved successfully!")
print("Vocabulary size:", len(vectorizer.vocabulary_))
print("Feature size:", X.shape[1])

TF-IDF vectorizer saved successfully!
Vocabulary size: 7193
Feature size: 7193


In [12]:
# for i in X[45]:
#     if i !=0 : print(f"{i}, ")
# analysing cell
for i in range(0,len(corpus)):
    if "c++" in corpus[i] and i > 1: 
        print(f"{i}")
        break
print(df['sentence'][i])
print(len([x for x in X[i] if x!=0]))
print(corpus[i])
print(df['importance_label'][i])


418
o    Solid programming knowledge (8+ years) with C/C++
4
solid program knowledg (8+ years) c/c++
1


In [13]:
# for i in X[116]:
#     if i !=0 : print(f"{i} ,")
# print(corpus[116])
# df['sentence'][116]
# analysing cell

In [14]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 
from sklearn.model_selection import train_test_split
y = df.iloc[:,1].values
print(y)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=0)


[0 0 1 ... 0 0 0]


In [15]:
y_train_pd = pd.Series(y_train)
y_train_pd.value_counts()

0    44021
1    25214
Name: count, dtype: int64

In [16]:
i = 124
print(len([x for x in X[i] if x != 0]),"[]",y_train[i])
# (df['sentence'][i].split())

Xtrain = torch.from_numpy(X_train).float()
Xtest = torch.from_numpy(X_test).float()
ytrain = torch.from_numpy(y_train)
ytest = torch.from_numpy(y_test)

Xtrain.shape , ytrain.shape


23 [] 0


(torch.Size([69235, 7193]), torch.Size([69235]))

In [22]:
input_size = 7193
output_siz = 2
hidden_size = 400

In [18]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 

In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = torch.nn.Linear(input_size,hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size,hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size,output_siz)
    
    def forward(self,X):
        X = torch.relu((self.fc1(X)))
        X = torch.relu((self.fc2(X)))
        X = self.fc3(X)
        return F.log_softmax(X,dim=1)

model = Net()

In [24]:
import torch.optim as optim
optimiser = torch.optim.Adam(model.parameters(),lr=0.01)
loss_fn = nn.NLLLoss()

In [25]:
epochs = 100
for epoch in range(epochs):
    optimiser.zero_grad()
    Ypred = model(Xtrain)
    loss = loss_fn(Ypred,ytrain)
    loss.backward()
    optimiser.step()
    print(f"Epoch {epoch} , loss {loss.item()}")
    

Epoch 0 , loss 0.6976122260093689
Epoch 1 , loss 0.6279799938201904
Epoch 2 , loss 0.5448691844940186
Epoch 3 , loss 0.48562532663345337
Epoch 4 , loss 0.44741424918174744
Epoch 5 , loss 0.421594500541687
Epoch 6 , loss 0.3921663165092468
Epoch 7 , loss 0.3654485046863556
Epoch 8 , loss 0.34575146436691284
Epoch 9 , loss 0.3242710828781128
Epoch 10 , loss 0.3069753348827362
Epoch 11 , loss 0.2889573872089386
Epoch 12 , loss 0.26912257075309753
Epoch 13 , loss 0.24830922484397888
Epoch 14 , loss 0.22533197700977325
Epoch 15 , loss 0.2027616947889328
Epoch 16 , loss 0.17997094988822937
Epoch 17 , loss 0.1583491414785385
Epoch 18 , loss 0.13819682598114014
Epoch 19 , loss 0.12075160443782806
Epoch 20 , loss 0.10626807063817978
Epoch 21 , loss 0.09448808431625366
Epoch 22 , loss 0.08430086821317673
Epoch 23 , loss 0.0747651606798172
Epoch 24 , loss 0.0657382383942604
Epoch 25 , loss 0.0572647824883461
Epoch 26 , loss 0.049251578748226166
Epoch 27 , loss 0.04190603271126747
Epoch 28 , loss 

In [26]:
# sample testing code for analysis
sample = """These
capabilities, together with our culture of shared success and commitment to creating 360° value, enable
us to help our clients succeed and build trusted, lasting relationships"""
sample = [process_sentence(sample)]
print(sample)
sample = vectorizer.transform(sample).toarray()
# now converting this sample to the tensor format for torch
sample = torch.from_numpy(sample).float()

sentiment = model(sample)
print(f"{sentiment} first sample")
sample = "Proficiency in Java, .Net, C#, C, Python, SQL, ReactJS, Angular across frontend/ backend databases." # negative 
sample = [process_sentence(sample)]
print(sample)
sample = vectorizer.transform(sample).toarray()
sample = torch.from_numpy(sample).float()
sentiment = model(sample)
print(f"{sentiment} second sample") # perfect prediction

# postive if second element is higher than first as prob of class 0 = exp(tnesor[0])
# else negative

['these capabilities, togeth cultur share success commit creat 360° value, enabl us help client succeed build trusted, last relationship']
tensor([[-1.0131e+01, -3.9815e-05]], grad_fn=<LogSoftmaxBackward0>) first sample
['profici java, .net, c#, c, python, sql, reactjs, angular across frontend/ backend databases.']
tensor([[-28.5136,   0.0000]], grad_fn=<LogSoftmaxBackward0>) second sample


In [27]:
# Evaluation

model.eval()

with torch.no_grad():
    y_pred = model(Xtest)                 # shape: (N, 2)
    y_labels = torch.argmax(y_pred, dim=1)
    correct = (y_labels == ytest).sum().item()

    accuracy = correct / ytest.size(0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

with torch.no_grad():
    y_pred = model(Xtest)
    test_loss = loss_fn(y_pred, ytest)

print(f"Test Loss: {test_loss.item():.4f}")

Test Accuracy: 91.44%
Test Loss: 1.2853


In [28]:
# saving the model
torch.save({
    "model_state": model.state_dict(),
    "optimizer_state": optimiser.state_dict()
}, "imp_sentence_classifier.pth")

Net(
  (fc1): Linear(in_features=7193, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=2, bias=True)
)